# Packaging Your Local Model for Deployment


In [ ]:
import tarfile

# Create model artifacts tar.gz file (SageMaker format)
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('trained_model.joblib')

# Uploading Model Artifacts to Amazon S3


In [ ]:
import sagemaker

# Create a SageMaker session
sagemaker_session = sagemaker.Session()

# Get the default bucket
default_bucket = sagemaker_session.default_bucket()

# Upload model to S3
model_artifact_uri = sagemaker_session.upload_data(
    path='model.tar.gz',
    bucket=default_bucket,
    key_prefix='models/local-trained'
)

# Print model artifacts URI
print(f"Model artifacts uploaded to: {model_artifact_uri}")

# Setting Up Configuration Variables


In [ ]:
# Get account id from the current AWS session
account_id = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']

# Define the SageMaker execution role ARN using the account id
SAGEMAKER_ROLE = f"arn:aws:iam::{account_id}:role/SageMakerDefaultExecution"

# Name for the SageMaker endpoint to be created
ENDPOINT_NAME = "california-housing-local-model-moein"

# Creating the Entry Point Script

model_fn function created in the entry_point.py file.

# Creating the SageMaker Model Object


In [ ]:
from sagemaker.sklearn.model import SKLearnModel

# Create a SKLearnModel from local model artifacts
model = SKLearnModel(
    model_data=model_artifact_uri,       # S3 location of model.tar.gz
    role=SAGEMAKER_ROLE,                 # IAM role for SageMaker
    entry_point='entry_point.py',        # Inference script
    framework_version='1.2-1',           # scikit-learn version
    py_version='py3',                    # Python version
    sagemaker_session=sagemaker_session  # SageMaker session
)

# Configuring Serverless Inference Settings


In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

# Configure serverless inference
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048,
    max_concurrency=10
)

# Deploying Your Model to a SageMaker Serverless Endpoint


In [ ]:
# Deploy model as serverless endpoint asynchronously
predictor = model.deploy(
    serverless_inference_config=serverless_config,
    endpoint_name=ENDPOINT_NAME,
    wait=False
)

# Retrieve detailed information about the specified SageMaker endpoint
endpoint_description = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)

# Extract the current status of the endpoint from the response
status = endpoint_description['EndpointStatus']

# Display the endpoint's status
print(f"Endpoint status: {status}")

# Connecting to Your SageMaker Endpoint


In [ ]:
from sagemaker.predictor import Predictor

# Connect to the deployed SageMaker endpoint for inference
predictor = Predictor(endpoint_name=ENDPOINT_NAME)
endpoint_context = predictor.endpoint_context()
properties = endpoint_context.properties
print(properties)

# Configuring Data Serialization


In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Set the serializer to convert input data (numpy arrays, pandas DataFrames) to CSV format for the endpoint
predictor.serializer = CSVSerializer()

# Set the deserializer to convert the CSV response from the endpoint back to Python objects (e.g., numpy arrays)
predictor.deserializer = CSVDeserializer()

# Evaluating Model Performance


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Path to local test dataset
TEST_DATA_FILE = "data/california_housing_test.csv"

# Load and split test data into features and target variable
test_df = pd.read_csv(TEST_DATA_FILE)
X_test = test_df.drop("MedHouseVal", axis=1)
y_test = test_df["MedHouseVal"]

# Make predictions on entire test set
predictions = predictor.predict(X_test.values)

# Evaluate the model
test_r2 = r2_score(y_test, predictions)
test_rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Print the results
print(f"R² Score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}")